In [1]:
import json
from core.utils import create_k_fold_splits

In [1]:
import os 

In [2]:
os.listdir('/home/ak/Music/')

[]

In [3]:
s = os.sep

In [7]:
sites = json.loads(open('test/inputspec.json').read())
sites = list(zip(range(len(sites)), sites))

In [18]:
for i, site in sites:
    site = f'local{i}'
    baseDir = f'test/input/{site}/simulatorRun/'
    data_dir = baseDir + s + f'subject_stats_S0{i}'
    splits_dir = baseDir + s + f'split_S0{i}'
    create_k_fold_splits(os.listdir(data_dir), 10, splits_dir)

In [14]:
'test/input/local0/simulatorRun/split_S00//'

'test/input/local0/simulatorRun/subject_stats_S00/'

In [4]:
!pip install --user docker

^C
ERROR: Operation cancelled by user
